In [2]:
import requests
#import urllib.request
#import time
from bs4 import BeautifulSoup
#from selenium import webdriver
import re
import pandas as pd

In [2]:
#routard
def scrap_routard(continent, country, city):
    """continent, country, city in FRENCH"""
    response = requests.get(f'https://www.routard.com/fr/guide/{continent}/{country}/{city}')
    routard_soup = BeautifulSoup(response.text, 'html.parser')
    links = routard_soup.find_all('a', attrs={'href': re.compile(f'guide/{continent}/{country}/{city}')})
    for link in links:
        print(link.get('href').split("/")[-1])
    # scraper les liens de chaque truc pour récup la description rapide ?

scrap_routard('europe', 'grece', 'athenes')

acropole
agora
erechtheion
jardin-national
mont-lycabette
musee-archeologique-national
musee-de-l-acropole
parlement-place-syndagma
parthenon
quartier-de-plaka


In [2]:
#guide michelin
region = 'attica'
city = 'athens'

In [9]:
COSTS = ['Affordable', 'Mid-Range', 'Premium', 'Luxury']

def scrap_michelin(city, region=None,country = None n_pages=1):
    # works if n_pages > number of pages on the website
    

    
    assert(n_pages>0), "n_pages below 0 !"
    data = []
    for i in range(n_pages):
        if i>0:
            url = base_url + f"/page/{i+1}?sort=distance"
        else:
            url = base_url + "?sort=distance"
        response = requests.get(url)
        michelin_soup = BeautifulSoup(response.text, 'html.parser')
        restaurants = michelin_soup.find_all('div', {'class': "card__menu-content"})
        if len(restaurants)==0:
            break
        for resto in restaurants:
            resto_a = resto.find('a')
            name = resto_a.text.strip()
            link = "https://guide.michelin.com" + resto_a.get('href')
            nb_stars = 0
            bib_gourmand = False #bib-gourmand: nos meilleurs rapports qualité-prix
            gastronomie_durable = False
            gastronomie_durable = False
            for span in resto.find_all('span', {'class':'distinction-icon'}):
                for distinction in span.find_all('img'):
                    src = distinction.get('src')
                    if "star" in src:
                        nb_stars += 1
                    elif "bib-gourmand" in src:
                        bib_gourmand = True
                    elif "gastronomie-durable" in src:
                        gastronomie_durable = True
    
            resto_div = resto.find_all('div', {'class': "card__menu-footer--score"})
            tmp = re.sub(r'\s+', ' ', resto_div[1].text).strip() # to get "<$/€> · <cuisine>"
            price, cuisine = tmp.split(" · ")
            cost = COSTS[len(price)-1]
            address, latitude, longitude = scrap_resto_link(link)
            data.append({
                "name": name,
                "link": link,
                "stars": nb_stars,
                "bib-gourmand": bib_gourmand,
                "cuisine": cuisine,
                "cost": cost,
                "address": address,
                "latitude":latitude,
                "longitude": longitude,
                "gastonomie_durable": gastronomie_durable
            })
    if len(data)==0:
        return None
    return pd.DataFrame(data)

def scrap_resto_link(url):
    resto_req = requests.get(url)
    resto_soup = BeautifulSoup(resto_req.text, 'html.parser')
    address = ""
    for div in resto_soup.find_all('div', {'class':'data-sheet__block--text'}):
        text = div.text.strip()
        if not '\n' in text:
            address = text
            break
    latitude, longitude = None, None
    for iframe in resto_soup.find_all('iframe'):
        i_src = iframe.get('src')
        if i_src is None:
            continue
        if "maps" in i_src:
            search = re.search(r'q=([-+]?[0-9]*\.?[0-9]+),([-+]?[0-9]*\.?[0-9]+)', i_src)
            if search:
                latitude = float(search.group(1))
                longitude = float(search.group(2))
    return address, latitude, longitude

# print(len(scrap_michelin('athens', 'attica')))
# print(len(scrap_michelin('athens', 'attica', 2)))

In [8]:
print(len(scrap_michelin('athens', 'attica', 3)))

36


In [17]:
# https://guide.michelin.com/fr/fr/osaka-region/restaurants
region = "osaka"
city = "osaka"
data = scrap_michelin(city, region, 2)
if data is None:
    data = scrap_michelin(region, 2)
data

In [14]:
# pattern urls:
#      same name region/city:
#      .../osaka-region/osaka/...
#      .../tokyo-region/tokyo/...
#      .../new-york-state/new-york/...
#      .../shanghai-municipality/shanghai/...
#      ...

#      region/city
#      .../attica/athens/...

#      france (region/city):
#      .../grand-est/reims/...

#      same city in us (idk other countries) (can't find index):
#      .../colorado/denver_1261491/...
#      .../denver_2892491_noindex/...
#      .../pennsylvania/denver_2942583_noindex/...

region = "osaka"
city = "osaka"

# possible implementation if you are known to be a fdp
data = scrap_michelin(city, region)
if data is None:
    data = scrap_michelin(region)
if data is None:
    data = scrap_michelin(city, f'{city}-region')
if data is None:
    data = scrap_michelin(city, f'{city}-state')
data

In [12]:
# pour ne pas montrer un écran noir mdt 5min avant que ça retourne quoi que ce soit pdt la prez
from tqdm import tqdm

for i in tqdm(range(10000000)):
    i+1

100%|█████████████████████████████████████████████████████████████████| 10000000/10000000 [00:03<00:00, 2983587.69it/s]


In [ ]:
# pour scrap plusieurs trucs en mm temps plutôt que de devoir attendre la fin d'un scrap pour en commencer un 2e
from threading import Thread

## Suppr stop words 13 langues

In [ ]:
!pip install langdetect

In [ ]:
from langdetect import detect
from nltk.corpus import stopwords
nltk.download('stopwords')

language_dict = {
  'cs': 'czech',
  'da': 'danish',
  'nl': 'dutch',
  'en':'english',
  'et': 'estonian',
  'fi': 'finnish',
  'fr': 'french',
  'de': 'german',
  'el': 'greek',
  'it': 'italian',
  'no': 'norwegian',
  'pl': 'polish',
  'pt': 'portuguese',
  'ru': 'russian',
  'sl': 'slovene',
  'es': 'spanish',
  'sv': 'swedish',
  'tr': 'turkish'
}


def tokenize_without_stopwords(text):
  try:
    language = language_dict[detect(text)]
    stop_words = set(stopwords.words(language))
    filtered_text = " ".join([word for word in text.split() if word not in stop_words])
    return word_tokenize(filtered_text, language=language)

  except Exception as e:
    if len(str(e)) == 4:
      print("[WARNING] language not supported:", str(e))
    else:
      print("error", e)
    return word_tokenize(text)

In [5]:
city = 'tokyo'
region = 'kanto'
pays = 'japon'
url = f"https://guide.michelin.com/fr/fr/restaurants?showMap=false&q={city}+{region}+{pays}"
response = requests.get(url)
michelin_soup = BeautifulSoup(response.text, 'html.parser')
restaurants = michelin_soup.find_all('div', {'class': "card__menu-content"})

In [7]:
len(restaurants)

20

In [22]:
def build_url(city, region, country, page=1):
    """
    Construit l'URL en fonction de la présence de city et country dans le DataFrame.
    """
    if city in df['city'].values and country in df['country'].values:
        base_url = df.loc[(df['city'] == city) & (df['country'] == country), 'url'].values[0]
        url = base_url + f"/page/{page}?sort=distance" if page > 1 else base_url + "?sort=distance"
    else:
        url = f"https://guide.michelin.com/fr/fr/restaurants?showMap=false&q={city}+{region if region else ''}+{country}"
    return url

def fetch_restaurant_data(url):
    """
    Récupère les données de restaurants depuis une URL donnée.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        restaurants = soup.find_all('div', {'class': "card__menu-content"})
        return restaurants if restaurants else None
    except requests.RequestException as e:
        print(f"Erreur lors de la requête : {e}")
        return None
    
def scrap_restaurant(restaurants):
    COSTS = ['Affordable', 'Mid-Range', 'Premium', 'Luxury']
    data = []
    for resto in restaurants:
        resto_a = resto.find('a')
        name = resto_a.text.strip()
        link = "https://guide.michelin.com" + resto_a.get('href')
        nb_stars = 0
        bib_gourmand = False #bib-gourmand: nos meilleurs rapports qualité-prix
        gastronomie_durable = False
        gastronomie_durable = False
        for span in resto.find_all('span', {'class':'distinction-icon'}):
            for distinction in span.find_all('img'):
                src = distinction.get('src')
                if "star" in src:
                    nb_stars += 1
                elif "bib-gourmand" in src:
                    bib_gourmand = True
                elif "gastronomie-durable" in src:
                    gastronomie_durable = True

        resto_div = resto.find_all('div', {'class': "card__menu-footer--score"})
        tmp = re.sub(r'\s+', ' ', resto_div[1].text).strip() # to get "<$/€> · <cuisine>"
        price, cuisine = tmp.split(" · ")
        cost = COSTS[len(price)-1]
        address, latitude, longitude = scrap_resto_link(link)
        data.append({
            "name": name,
            "link": link,
            "stars": nb_stars,
            "bib-gourmand": bib_gourmand,
            "cuisine": cuisine,
            "cost": cost,
            "address": address,
            "latitude":latitude,
            "longitude": longitude,
            "gastonomie_durable": gastronomie_durable
        })
    if len(data)==0:
        return None
    return pd.DataFrame(data)


def scrap_resto_link(url):
    resto_req = requests.get(url)
    resto_soup = BeautifulSoup(resto_req.text, 'html.parser')
    address = ""
    for div in resto_soup.find_all('div', {'class':'data-sheet__block--text'}):
        text = div.text.strip()
        if not '\n' in text:
            address = text
            break
    latitude, longitude = None, None
    for iframe in resto_soup.find_all('iframe'):
        i_src = iframe.get('src')
        if i_src is None:
            continue
        if "maps" in i_src:
            search = re.search(r'q=([-+]?[0-9]*\.?[0-9]+),([-+]?[0-9]*\.?[0-9]+)', i_src)
            if search:
                latitude = float(search.group(1))
                longitude = float(search.group(2))
    return address, latitude, longitude

def scrap_michelin(city, region=None, country=None, n_pages=3):
    """
    Fonction principale pour récupérer les informations des restaurants d'une ville.
    Retourne un DataFrame consolidé avec toutes les pages scrappées.
    """
    df = pd.read_csv('links_michelin_norm.csv')
    assert n_pages > 0, "n_pages doit être supérieur à 0 !"

    # Initialisation d'un DataFrame vide avec les bonnes colonnes
    df_restaurants = pd.DataFrame(columns=["name", "link", "stars", "bib-gourmand", 
                                           "cuisine", "cost", "address", "latitude", 
                                           "longitude", "gastonomie_durable"])

    page = 1

    while page <= n_pages:
        if city in df['city'].values and country in df['country'].values:
            base_url = df.loc[(df['city'] == city) & (df['country'] == country), 'url'].values[0]
            url = base_url + f"/page/{page}?sort=distance" if page > 1 else base_url + "?sort=distance"
        else:
            url = f"https://guide.michelin.com/fr/fr/restaurants?showMap=false&q={city}+{region if region else ''}+{country}"
        restaurants = fetch_restaurant_data(url)

        if not restaurants:
            break  # Stopper si aucun restaurant trouvé

        new_data = scrap_restaurant(restaurants)
        
        if new_data is not None and not new_data.empty:
            df_restaurants = pd.concat([df_restaurants, new_data], ignore_index=True)

        page += 1
        
        if len(restaurants) < 20:
            break  # Arrêter si moins de 20 restaurants trouvés (fin de la liste)

    return df_restaurants



In [23]:
df = pd.read_csv('links_michelin_norm.csv')
df.head()

,country,city,region_url,city_url,url
0,india,pune,grand-est,punerot_1543607_noindex,https://guide.michelin.com/fr/fr/grand-est/pun...
1,india,pune,canarias,puerto-de-la-cruz,https://guide.michelin.com/fr/fr/canarias/puer...
2,india,pune,andalucia,el-puerto-de-santa-maria,https://guide.michelin.com/fr/fr/andalucia/el-...
3,india,surat,cchanghwad-suraasdrthaanii,restaurants,https://guide.michelin.com/fr/fr/cchanghwad-su...
4,india,kanpur,noord-holland,purmerend,https://guide.michelin.com/fr/fr/noord-holland...


In [29]:
scrap_michelin('bangkok','','thailande',2)

C:\Users\marin\AppData\Local\Temp\ipykernel_20904\2256401209.py:119: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_restaurants = pd.concat([df_restaurants, new_data], ignore_index=True)


,name,link,stars,bib-gourmand,cuisine,cost,address,latitude,longitude,gastonomie_durable
0,Lucky Seafood,https://guide.michelin.com/fr/fr/bangkok-regio...,0,True,Poissons et fruits de mer,Mid-Range,"28/1 Phran Nok-Phutthamonthon 4 Road, Bang Phr...",13.753559,100.455705,False
1,GOAT,https://guide.michelin.com/fr/fr/bangkok-regio...,1,False,Thaïlandaise contemporaine,Premium,"12 Soi Ekkamai 10 Lane 2, Khlong Tan Nuea, Vad...",13.727437,100.587236,False
2,Kinu by Takagi,https://guide.michelin.com/fr/fr/bangkok-regio...,0,False,Cuisine japonaise,Luxury,"2F, Mandarin Oriental Hotel, 48 Soi Charoen Kr...",13.723940,100.515700,False
3,Jamn,https://guide.michelin.com/fr/fr/bangkok-regio...,0,False,Cuisine thaïlandaise,Premium,"8 Soi Chan 43 Yaek 41, Bang Khlo, Bang Kho Lae...",13.700079,100.527844,False
4,Palmier by Guillaume Galliot,https://guide.michelin.com/fr/fr/bangkok-regio...,0,False,Cuisine française,Premium,,13.712179,100.510008,False
5,Gaa,https://guide.michelin.com/fr/fr/bangkok-regio...,2,False,Cuisine indienne,Luxury,"46/1 Sukhumvit 53, Khlong Tan Nuea, Vadhana, B...",13.727250,100.578280,False
6,Janhom,https://guide.michelin.com/fr/fr/bangkok-regio...,0,True,de Thaïlande du Sud,Affordable,"273/4 Soi Ramkhamhaeng 21, Ramkhamhaeng Road, ...",13.761241,100.602787,False
7,Chop Chop Cook Shop,https://guide.michelin.com/fr/fr/bangkok-regio...,0,False,Chino-thaïlandaise,Mid-Range,"328 Yaowarat Road, Chakkrawat, Samphanthawong,...",13.741438,100.507698,False
8,Sushi Saito,https://guide.michelin.com/fr/fr/bangkok-regio...,0,False,Sushi,Luxury,"Four Seasons Private Residences, 300 Charoen K...",13.712541,100.510048,False
9,Bangkok Bold (Pathum Wan),https://guide.michelin.com/fr/fr/bangkok-regio...,0,False,Cuisine thaïlandaise,Mid-Range,"LGF, Central Embassy, 1031 Phloen Chit Road, L...",13.744191,100.546086,False
